In [31]:
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge

In [2]:
data_train = pd.read_csv('salary-train.csv')
data_test = pd.read_csv('salary-test.csv')

In [18]:
data_train['ContractTime'].fillna('nan', inplace=True)

data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['LocationNormalized'] = data_train['LocationNormalized'].str.lower()

data_train['FullDescription'] = data_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
data_train['FullDescription'] = data_train['FullDescription'].str.lower()

In [28]:
vectorizer = TfidfVectorizer(min_df=5)
X_train_desc = vectorizer.fit_transform(data_train['FullDescription'])
X_test_desc = vectorizer.transform(data_test['FullDescription'])

enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [54]:
X_train = sc.sparse.hstack((X_train_desc, X_train_categ))
Y_train = data_train['SalaryNormalized']

X_test = sc.sparse.hstack((X_test_desc, X_test_categ))

In [55]:
clf = Ridge(alpha=1.0, random_state=241)
clf.fit(X_train, Y_train) 

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [61]:
answer = clf.predict(X_test)
answer = " ".join(["%.2f" % x for x in answer])

with open('answer.txt', 'w') as f:
    f.write(answer)
    
answer

'55850.59 37182.51'